## EKF Localisation using UTIAS dataset with known correlation

### Importing packages and other helper files

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from helper_functions import *

### Load and resample raw data from UTIAS data set

In [2]:
#The robot's position groundtruth, odometry, and measurements are stored in Robots
Barcodes, Landmark_Groundtruth, Robots = load_MRCLAM_dataSet();
Barcodes = Barcodes.astype(int)
codedict = dict(np.column_stack((Barcodes[:,1], Barcodes[:,0])))

Loading data
Reading robot1_Groundtruth data
Reading robot1_odometry data
Reading robot1_measurements data
Robot1 data read successfully
------------------------------------------------
All data successfully read
------------------------------------------------


In [3]:
Robots, timesteps = sample_MRCLAM_dataSet(Robots)

### Defining constants for the simulation

In [4]:
deltaT = 0.02; 

#robot-dependent motion noise parameters, see equation 3
alphas = [0.2, 0.03, 0.09, 0.08, 0, 0];  

#robot-dependent sensor noise parameters, see equation 9
sigma_range = 2;
sigma_bearing = 3;
sigma_id = 1;
initial_measurement_variance = 0.01  #very

Q_t = np.array([
        [sigma_range**2, 0, 0],
        [0, sigma_bearing**2, 0],
        [0, 0, sigma_id**2]
        ])

measurement_prob = 0;
n_robots = 1;
robot_num = 1;

### Initialise estimated parameters to zeros

In [5]:
# add pose estimate matrix to Robots
# data will be added to this as the program runs
Robots[str(robot_num)]['Est'] = np.zeros((Robots[str(robot_num)]['G'].shape[0], 4))


# initialize time, and pose estimate
# start index is set to 600 because earlier data was found to cause problems
start = 600; 
start = start -1 #due to python indexing

# set start time
t = Robots[str(robot_num)]['G'][start, 0]

# set starting pose mean to pose groundtruth at start time
poseMean = Robots[str(robot_num)]['G'][start,1:4].reshape(3,1)
poseCov = np.ones((3,3)) *  initial_measurement_variance

#tracks which measurement is next received
#iterating to measurement index which time corresponds to 
measurementIndex = 0;

#set up map between barcodes and landmark IDs
#codeDict = containers.Map(Barcodes(:,2),Barcodes(:,1));

#advance measurement index until the next measurement time is greater than the starting time
while (Robots[str(robot_num)]['M'][measurementIndex, 0] < (t - 0.05)):
        measurementIndex = measurementIndex + 1;

### Running EKF on the dataset

In [6]:
"""
loop through all odometry and measurement samples
updating the robot's pose estimate with each step
reference table 7.2 in Probabilistic Robotics
"""

for i in range(start, Robots[str(robot_num)]['G'].shape[0]):

    theta = poseMean[2, 0];
    #update time
    t = Robots[str(robot_num)]['G'][i,0]

    #update movement vector per equation 1
    u_t = np.array([Robots[str(robot_num)]['O'][i,1], Robots[str(robot_num)]['O'][i,2]]);

    rot = deltaT * u_t[1];
    halfRot = rot / 2;
    trans = u_t[0] * deltaT;

    #calculate the movement Jacobian per equation 2
    G_t = np.array([
                    [1, 0, trans * -np.sin(theta + halfRot)],
                    [0, 1, trans *  np.cos(theta + halfRot)],
                    [0, 0, 1]
                    ])

    #calculate motion covariance in control space per equation 3
    M_t = np.array([
                    [(alphas[0] * abs(u_t[0]) + alphas[1] * abs(u_t[1]))**2, 0],
                    [0, (alphas[2] * abs(u_t[1]) + alphas[3] * abs(u_t[1]))**2]
                    ])


    #calculate Jacobian to transform motion covariance to state space per equation 4
    V_t = np.array([
                    [np.cos(theta + halfRot), -0.5 * np.sin(theta + halfRot)],
                    [np.sin(theta + halfRot),  0.5 * np.cos(theta + halfRot)],
                    [0, 1]
                    ])
    
    #calculate pose update
    poseUpdate = np.array([trans * np.cos(theta + halfRot), trans * np.sin(theta + halfRot), rot]).reshape(3,1)
    
    #calculate estimated pose mean per equation 1
    poseMeanBar = poseMean + poseUpdate;
    #print('poseMeanBar is ' + str(poseMeanBar))

    #calculate estimated pose covariance per equation 5
    poseCovBar = np.dot(G_t, np.dot(poseCov, G_t.T)) + np.dot(V_t, np.dot(M_t, V_t.T))
    #print('poseCovBar is ' + str(poseCovBar))
    
    #get measurements for the current timestep, if any exist
    z, measurementIndex = getObservations(Robots, robot_num, t, measurementIndex, codedict)
    
    #create two matrices for expected measurement and measurement covariance
    S    = np.zeros((z.shape[1],3,3))
    zHat = np.zeros((3, z.shape[1]))

    #if any measurements are available
    if z[2,0] > 1:
        for k in range(0, z.shape[1]): #loop over every measurement
            j = int(z[2,k]);

            #get coordinates of the measured landmark
            m = Landmark_Groundtruth[j-1, 1:3];

            #compute the expected measurement per equations 6 and 7
            xDist = (m[0] - poseMeanBar[0]).item(); #convert the np float to python float type for operations
            yDist = (m[1] - poseMeanBar[1]).item();
            q = np.power(xDist,2) + np.power(yDist,2)
            

            #constrains expected bearing to between 0 and 2*pi
            temp = np.arctan2(yDist, xDist) 
            pred_bear = conBear(temp - poseMeanBar[2]);                
            zHat[:,k] = np.array([np.sqrt(q), pred_bear, j]);

            #calculate Jacobian of the measurement model per equation 8
            H = np.array([
                            [(-1 * (xDist / np.sqrt(q))), (-1 * (yDist / np.sqrt(q))), 0],
                            [(yDist / q), (-1 * (xDist / q)), -1],
                            [0, 0, 0]
                        ])

            #compute S per equation 9
            S[k,:,:] = np.dot(H, np.dot(poseCovBar, H.T)) + Q_t;

            #compute Kalman gain per equation 10
            K = np.dot(poseCov, np.dot(H.T, np.linalg.inv(S[k,:,:])));

            #update pose mean and covariance estimates per equations 11 and 12            
            innovation = (z[:,k] - zHat[:,k]).reshape(3,1)
            
            poseMeanBar = poseMeanBar + np.dot(K, innovation);            
            poseCovBar  = np.dot((np.eye(3) - np.dot(K,H)), poseCovBar);
            
    #update pose mean and covariance
    #constrains heading to between 0 and 2*pi
    poseMean = poseMeanBar;
    poseMean[2] = conBear(poseMean[2]);
    poseCov = poseCovBar;

    #add pose mean to estimated position vector
    Robots[str(robot_num)]['Est'][i,:] = np.array([t, poseMean[0], poseMean[1], poseMean[2]])
    
    if i % 10000 ==0:
        print(str(i) + 'th iteration done')
print('--------------------------------------')

10000th iteration done
20000th iteration done
30000th iteration done
40000th iteration done
50000th iteration done
60000th iteration done
70000th iteration done
--------------------------------------


### Checking the RMSE on the estimation

In [7]:
print('Total deviation is ' + str(path_loss(Robots, robot_num, 600)))

Total deviation is 198918.6097448089
